## 데이터 불러오기

In [1]:
### 구글 서버와 내 드라이브 연결하기
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
### 작업 디렉토리 변경하기
%cd '/content/drive/MyDrive/KDT/비정형텍스트분석'

/content/drive/MyDrive/KDT/비정형텍스트분석


In [3]:
### konlpy, Mecab 형태소 분석기 설치 스크립트 실행 - 설치하는데 2분 10초 ~ 20초 
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

--2023-05-26 02:05:19--  https://www.dropbox.com/s/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.2.18, 2620:100:6022:18::a27d:4212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.2.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz [following]
--2023-05-26 02:05:20--  https://www.dropbox.com/s/dl/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucaa4588e8060538bae16ca1726e.dl.dropboxusercontent.com/cd/0/get/B8xvCD6l9IFPGXkQ_DevYA-Ck7WC5lDDGy8ftqMnOP83XDUgbgsIyk-KQLGnuf-urDWg85_HtHjxKOAFFfBsB3mSCVWoP5LC2egyhh5Id8z1SJpZT81vS7cHmocqWvBI1EhXLovhG-ZyQ_Ez8vINr-MmSGyOa8t27AHHo100s2bGGnZchG9ak_2PD0PxxWV0aTE/file?dl=1# [following]
--2023-05-26 02:05:20--  https://ucaa4588e8060538bae16ca1726e.dl.dropboxusercontent.com/cd/0/get/B8xvCD6l9IF

In [4]:
### 필요한 라이브러리 임폴트
import os
import re
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
from konlpy.tag import Okt

In [5]:
### 파일 경로 설정
file_path = 'ChatBotData.csv_short'

In [6]:
### pd.read_csv() --> DataFrame 생성
df = pd.read_csv(file_path, encoding='utf-8')

# 결과 확인하기
df

,Q,A,label
0,가끔 궁금해,그 사람도 그럴 거예요.,0
1,가끔 뭐하는지 궁금해,그 사람도 그럴 거예요.,0
2,가끔은 혼자인게 좋다,혼자를 즐기세요.,0
3,가난한 자의 설움,돈은 다시 들어올 거예요.,0
4,가만 있어도 땀난다,땀을 식혀주세요.,0
5,가상화폐 쫄딱 망함,어서 잊고 새출발 하세요.,0
6,가스불 켜고 나갔어,빨리 집에 돌아가서 끄고 나오세요.,0
7,가스불 켜놓고 나온거 같아,빨리 집에 돌아가서 끄고 나오세요.,0
8,가스비 너무 많이 나왔다.,다음 달에는 더 절약해봐요.,0
9,가스비 비싼데 감기 걸리겠어,따뜻하게 사세요!,0


## 데이터 전처리

In [7]:
### 누락 데이터 확인
num_nulls = df.isnull().sum()

# 결과 확인하기
print(num_nulls)

Q        0
A        0
label    0
dtype: int64


In [8]:
### 정규 표현식 --> 문장 부호 제거

# 정규 표현식 패턴 생성
pattern = "[!?.,~\"';:)(]"

# 질문 컬럼 --> 문장 부호 제거 --> 리스트 자료형 생성
sentences_Q = list(df['Q'].apply(lambda x:re.sub(pattern, "", x)))
print(f'결과 확인하기 : \n{sentences_Q}')

print('-'*80)

# 답변 컬럼 --> 문장 부호 제거 --> 리스트 자료형 생성
sentences_A = list(df['A'].apply(lambda x:re.sub(pattern, "", x)))
print(f'결과 확인하기 : \n{sentences_A}')

결과 확인하기 : 
['가끔 궁금해', '가끔 뭐하는지 궁금해', '가끔은 혼자인게 좋다', '가난한 자의 설움', '가만 있어도 땀난다', '가상화폐 쫄딱 망함', '가스불 켜고 나갔어', '가스불 켜놓고 나온거 같아', '가스비 너무 많이 나왔다', '가스비 비싼데 감기 걸리겠어', '남자친구 교회 데려가고 싶어', '남자친구 또 운동 갔어', '남자친구 생일인데 뭘 줄까', '남자친구 승진 선물로 뭐가 좋을까', '남자친구 오늘 따라 훈훈해 보인다', '남자친구 오늘 좀 질린다', '남자친구가 나 안 믿어줘', '남자친구가 너무 바빠', '남자친구가 너무 운동만 해', '남자친구가 너무 잘생겼어']
--------------------------------------------------------------------------------
결과 확인하기 : 
['그 사람도 그럴 거예요', '그 사람도 그럴 거예요', '혼자를 즐기세요', '돈은 다시 들어올 거예요', '땀을 식혀주세요', '어서 잊고 새출발 하세요', '빨리 집에 돌아가서 끄고 나오세요', '빨리 집에 돌아가서 끄고 나오세요', '다음 달에는 더 절약해봐요', '따뜻하게 사세요', '마음을 열 때까지 설득해보세요', '운동을 함께 해보세요', '평소에 필요한 것 생각해보세요', '평소에 필요했던 게 좋을 것 같아요', '전생에 나라를 구하셨나요', '결단은 빠를수록 좋아요', '거짓말 적당히 하세요', '너무 집착하지 마세요', '운동을 함께 해보세요', '전생에 나라를 구하셨나요']


## 데이터 분석

In [9]:
### Okt 형태소 분석기 --> 질문과 답변 문장 --> 형태소 분할

# 형태소 분석기 객체 생성
okt = Okt()

# 질문 문장 --> 형태소 분할
morphs_Q = [okt.morphs(sentence) for sentence in sentences_Q]
print(f'형태소 분석의 결과 : \n{morphs_Q[:3]}')

print('-'*80)

# 답변 문장 --> 형태소 분할
morphs_A = [okt.morphs(sentence) for sentence in sentences_A]
print(f'형태소 분석의 결과 : \n{morphs_A[:3]}')

형태소 분석의 결과 : 
[['가끔', '궁금해'], ['가끔', '뭐', '하는지', '궁금해'], ['가끔', '은', '혼자', '인게', '좋다']]
--------------------------------------------------------------------------------
형태소 분석의 결과 : 
[['그', '사람', '도', '그럴', '거', '예요'], ['그', '사람', '도', '그럴', '거', '예요'], ['혼자', '를', '즐기세요']]


In [10]:
### 형태소 분할의 결과 --> 문장의 길이 추출

# 질문 문장
len_morphs_Q = [len(sentence) for sentence in morphs_Q]
print(f'문장의 길이 추출 결과 : \n{len_morphs_Q}')

print('-'*80)

# 답변 문장
len_morphs_A = [len(sentence) for sentence in morphs_A]
print(f'문장의 길이 추출 결과 : \n{len_morphs_A}')

문장의 길이 추출 결과 : 
[2, 4, 5, 3, 4, 4, 4, 5, 5, 5, 4, 4, 5, 7, 5, 4, 5, 4, 6, 4]
--------------------------------------------------------------------------------
문장의 길이 추출 결과 : 
[6, 6, 3, 6, 3, 5, 6, 6, 6, 2, 7, 4, 6, 7, 6, 4, 3, 4, 4, 6]


In [11]:
### 형태소 분할의 결과 --> 문장의 길이에 대한 통계량 추출

## 질문 문장
# 최대값
max = np.max(len_morphs_Q)
# 최소값
min = np.min(len_morphs_Q)
# 평균값
mean = np.mean(len_morphs_Q)
# 중간값
median = np.median(len_morphs_Q)
# 1사분위 값
q1 = np.percentile(len_morphs_Q, q=25)
# 3사분위 값
q3 = np.percentile(len_morphs_Q, q=75)

# 결과 확인하기
print(f'질문 문장의 단어 개수 최대값 : {max}')
print(f'질문 문장의 단어 개수 최소값 : {min}')
print(f'질문 문장의 단어 개수 평균값 : {mean}')
print(f'질문 문장의 단어 개수 중간값 : {median}')
print(f'질문 문장의 단어 개수 1사분위 값 : {q1}')
print(f'질문 문장의 단어 개수 3사분위 값 : {q3}')

print('-'*80)

## 답변 문장
# 최대값
max = np.max(len_morphs_A)
# 최소값
min = np.min(len_morphs_A)
# 평균값
mean = np.mean(len_morphs_A)
# 중간값
median = np.median(len_morphs_A)
# 1사분위 값
q1 = np.percentile(len_morphs_A, q=25)
# 3사분위 값
q3 = np.percentile(len_morphs_A, q=75)

# 결과 확인하기
print(f'답변 문장의 단어 개수 최대값 : {max}')
print(f'답변 문장의 단어 개수 최소값 : {min}')
print(f'답변 문장의 단어 개수 평균값 : {mean}')
print(f'답변 문장의 단어 개수 중간값 : {median}')
print(f'답변 문장의 단어 개수 1사분위 값 : {q1}')
print(f'답변 문장의 단어 개수 3사분위 값 : {q3}')

질문 문장의 단어 개수 최대값 : 7
질문 문장의 단어 개수 최소값 : 2
질문 문장의 단어 개수 평균값 : 4.45
질문 문장의 단어 개수 중간값 : 4.0
질문 문장의 단어 개수 1사분위 값 : 4.0
질문 문장의 단어 개수 3사분위 값 : 5.0
--------------------------------------------------------------------------------
답변 문장의 단어 개수 최대값 : 7
답변 문장의 단어 개수 최소값 : 2
답변 문장의 단어 개수 평균값 : 5.0
답변 문장의 단어 개수 중간값 : 6.0
답변 문장의 단어 개수 1사분위 값 : 4.0
답변 문장의 단어 개수 3사분위 값 : 6.0


## 전처리 함수 확인

In [12]:
### 데이터 불러오기
def load_data(path):
    # pd.read_csv() --> DataFrame 생성
    df = pd.read_csv(path, encoding='utf-8')
    # 질문과 답변 컬럼을 가져와서 리스트 자료형 생성 
    question = list(df['Q'])
    answer = list(df['A'])
    return question, answer 

In [13]:
##### 단어 사전 생성하기

def load_vocabulary(path, vocab_path, tokenize_as_morph=False):
    # 사전을 담을 배열 준비
    vocabulary_list = []
    # 사전을 구성한 후 파일로 저장한다.
    # 그 파일의 존재 유무를 확인하다.
    if not os.path.exists(vocab_path):
        # 이미 생성된 사전 파일이 존재하지 않으므로
        # 데이터를 가지고 만들어야한다.
        # 그래서 데이터 파일의 존재 유무를 확인한다.
        if os.path.exists(path):
            # 데이터가 존재하므로 pd.read_csv() 함수를 이용
            # 데이터프레임 생성
            data_df = pd.read_csv(path, encoding='utf-8')
            # 질문 컬럼과 답변 컬럼을 인덱싱한다
            question = list(data_df['Q'])
            answer = list(data_df['A'])
            # 형태소 분석기로 형태소로 분할
            if tokenize_as_morph:
                question = sentences_tokenized_morphs(question)
                answer = sentences_tokenized_morphs(answer)
            data = []
            # 질문과 답변을 extend 함수를 이용, 하나의 리스트로 병합 
            data.extend(question)
            data.extend(answer)
            # 토큰(형태소)으로 분할
            words = data_tokenizer(data)
            # 중복을 제거하기 위해서 set 자료형으로 만들어주고,
            # 다시 리스트로 만들어준다.
            words = list(set(words))
            # words(형태소 리스트)에 특수 토큰 추가 
            words[:0] = MARKER
        with open(vocab_path, 'w', encoding='utf-8') as vocabulary_file:
            for word in words:
                vocabulary_file.write(word+'\n')
    # vocabulary.txt 파일이 존재하면 해당 파일을 불러서 
    # 리스트에 그 값을 넣어준다.
    with open(vocab_path, 'r', encoding='utf-8') as vocabulary_file:
        for line in vocabulary_file:
            vocabulary_list.append(line.strip())
    # 키와 값이 존재하는 디셔너리 생성
    word2idx, idx2word = make_vocabulary(vocabulary_list)
    return word2idx, idx2word, len(word2idx)

In [14]:
### load_data() 함수 실행
question, answer = load_data(file_path)
print(question)
print(answer)

['가끔 궁금해', '가끔 뭐하는지 궁금해', '가끔은 혼자인게 좋다', '가난한 자의 설움', '가만 있어도 땀난다', '가상화폐 쫄딱 망함', '가스불 켜고 나갔어', '가스불 켜놓고 나온거 같아', '가스비 너무 많이 나왔다.', '가스비 비싼데 감기 걸리겠어', '남자친구 교회 데려가고 싶어', '남자친구 또 운동 갔어', '남자친구 생일인데 뭘 줄까', '남자친구 승진 선물로 뭐가 좋을까?', '남자친구 오늘 따라 훈훈해 보인다', '남자친구 오늘 좀 질린다.', '남자친구가 나 안 믿어줘', '남자친구가 너무 바빠', '남자친구가 너무 운동만 해', '남자친구가 너무 잘생겼어']
['그 사람도 그럴 거예요.', '그 사람도 그럴 거예요.', '혼자를 즐기세요.', '돈은 다시 들어올 거예요.', '땀을 식혀주세요.', '어서 잊고 새출발 하세요.', '빨리 집에 돌아가서 끄고 나오세요.', '빨리 집에 돌아가서 끄고 나오세요.', '다음 달에는 더 절약해봐요.', '따뜻하게 사세요!', '마음을 열 때까지 설득해보세요.', '운동을 함께 해보세요.', '평소에 필요한 것 생각해보세요.', '평소에 필요했던 게 좋을 것 같아요.', '전생에 나라를 구하셨나요.', '결단은 빠를수록 좋아요.', '거짓말 적당히 하세요.', '너무 집착하지 마세요.', '운동을 함께 해보세요.', '전생에 나라를 구하셨나요.']


In [15]:
### Okt 형태소 분석기 --> 토큰화 --> 문장 재생성 --> 리스트 데이터 생성
def sentences_tokenized_morphs(data):
    okt = Okt()
    result_data = []
    for seq in tqdm(data):
        morphlized_seq = " ".join(okt.morphs(seq))
        result_data.append(morphlized_seq)

    return result_data

In [16]:
### sentences_tokenized_morphs() 함수 실행
question_sentences_list = sentences_tokenized_morphs(question)
answer_sentences_list = sentences_tokenized_morphs(answer)

# 결과 확인하기
print(question_sentences_list)
print('-'*80)
print(answer_sentences_list)

100%|██████████| 20/20 [00:00<00:00, 82.27it/s] 

['가끔 궁금해', '가끔 뭐 하는지 궁금해', '가끔 은 혼자 인게 좋다', '가난한 자의 설움', '가만 있어도 땀 난다', '가상 화폐 쫄딱 망함', '가스 불 켜고 나갔어', '가스 불 켜놓고 나온거 같아', '가스 비 너무 많이 나왔다 .', '가스 비 비싼데 감기 걸리겠어', '남자친구 교회 데려가고 싶어', '남자친구 또 운동 갔어', '남자친구 생일 인데 뭘 줄까', '남자친구 승진 선물 로 뭐 가 좋을까 ?', '남자친구 오늘 따라 훈훈해 보인다', '남자친구 오늘 좀 질린다 .', '남자친구 가 나 안 믿어줘', '남자친구 가 너무 바빠', '남자친구 가 너무 운동 만 해', '남자친구 가 너무 잘생겼어']
--------------------------------------------------------------------------------
['그 사람 도 그럴 거 예요 .', '그 사람 도 그럴 거 예요 .', '혼자 를 즐기세요 .', '돈 은 다시 들어올 거 예요 .', '땀 을 식혀주세요 .', '어서 잊고 새 출발 하세요 .', '빨리 집 에 돌아가서 끄고 나오세요 .', '빨리 집 에 돌아가서 끄고 나오세요 .', '다음 달 에는 더 절약 해봐요 .', '따뜻하게 사세요 !', '마음 을 열 때 까지 설득 해보세요 .', '운동 을 함께 해보세요 .', '평소 에 필요한 것 생각 해보세요 .', '평소 에 필요했던 게 좋을 것 같아요 .', '전생 에 나라 를 구 하셨나요 .', '결단 은 빠를수록 좋아요 .', '거짓말 적당히 하세요 .', '너무 집착 하지 마세요 .', '운동 을 함께 해보세요 .', '전생 에 나라 를 구 하셨나요 .']


In [17]:
### 질문과 답변 --> 하나의 리스트로 병합
data = []
data.extend(question_sentences_list)
data.extend(answer_sentences_list)

# 결과 확인하기
print(data)

['가끔 궁금해', '가끔 뭐 하는지 궁금해', '가끔 은 혼자 인게 좋다', '가난한 자의 설움', '가만 있어도 땀 난다', '가상 화폐 쫄딱 망함', '가스 불 켜고 나갔어', '가스 불 켜놓고 나온거 같아', '가스 비 너무 많이 나왔다 .', '가스 비 비싼데 감기 걸리겠어', '남자친구 교회 데려가고 싶어', '남자친구 또 운동 갔어', '남자친구 생일 인데 뭘 줄까', '남자친구 승진 선물 로 뭐 가 좋을까 ?', '남자친구 오늘 따라 훈훈해 보인다', '남자친구 오늘 좀 질린다 .', '남자친구 가 나 안 믿어줘', '남자친구 가 너무 바빠', '남자친구 가 너무 운동 만 해', '남자친구 가 너무 잘생겼어', '그 사람 도 그럴 거 예요 .', '그 사람 도 그럴 거 예요 .', '혼자 를 즐기세요 .', '돈 은 다시 들어올 거 예요 .', '땀 을 식혀주세요 .', '어서 잊고 새 출발 하세요 .', '빨리 집 에 돌아가서 끄고 나오세요 .', '빨리 집 에 돌아가서 끄고 나오세요 .', '다음 달 에는 더 절약 해봐요 .', '따뜻하게 사세요 !', '마음 을 열 때 까지 설득 해보세요 .', '운동 을 함께 해보세요 .', '평소 에 필요한 것 생각 해보세요 .', '평소 에 필요했던 게 좋을 것 같아요 .', '전생 에 나라 를 구 하셨나요 .', '결단 은 빠를수록 좋아요 .', '거짓말 적당히 하세요 .', '너무 집착 하지 마세요 .', '운동 을 함께 해보세요 .', '전생 에 나라 를 구 하셨나요 .']


In [18]:
### 형태소로 분석(분할)한 문장 --> 문장 부호 제거 --> 공백 단위로 분할 --> 형태소의 리스트 생성
def data_tokenizer(data):
    # 공백 단위로 분할해서 형태소(단어)를 담을 리스트 생성
    words = []
    for sentence in data:
        # 정규 표현식 패턴 생성
        pattern = "[~.,?!\"':;)(]"
        # 정규 표현식을 이용 --> 문장 부호를 모두 삭제("")
        sentence = re.sub(pattern, "", sentence)
        for word in sentence.split():
            words.append(word)
    return words

In [19]:
### data_tokenizer() 함수 실행
words = data_tokenizer(data)

# 결과 확인하기
print(words)

['가끔', '궁금해', '가끔', '뭐', '하는지', '궁금해', '가끔', '은', '혼자', '인게', '좋다', '가난한', '자의', '설움', '가만', '있어도', '땀', '난다', '가상', '화폐', '쫄딱', '망함', '가스', '불', '켜고', '나갔어', '가스', '불', '켜놓고', '나온거', '같아', '가스', '비', '너무', '많이', '나왔다', '가스', '비', '비싼데', '감기', '걸리겠어', '남자친구', '교회', '데려가고', '싶어', '남자친구', '또', '운동', '갔어', '남자친구', '생일', '인데', '뭘', '줄까', '남자친구', '승진', '선물', '로', '뭐', '가', '좋을까', '남자친구', '오늘', '따라', '훈훈해', '보인다', '남자친구', '오늘', '좀', '질린다', '남자친구', '가', '나', '안', '믿어줘', '남자친구', '가', '너무', '바빠', '남자친구', '가', '너무', '운동', '만', '해', '남자친구', '가', '너무', '잘생겼어', '그', '사람', '도', '그럴', '거', '예요', '그', '사람', '도', '그럴', '거', '예요', '혼자', '를', '즐기세요', '돈', '은', '다시', '들어올', '거', '예요', '땀', '을', '식혀주세요', '어서', '잊고', '새', '출발', '하세요', '빨리', '집', '에', '돌아가서', '끄고', '나오세요', '빨리', '집', '에', '돌아가서', '끄고', '나오세요', '다음', '달', '에는', '더', '절약', '해봐요', '따뜻하게', '사세요', '마음', '을', '열', '때', '까지', '설득', '해보세요', '운동', '을', '함께', '해보세요', '평소', '에', '필요한', '것', '생각', '해보세요', '평소', '에', '필요했던', '게', '좋을', '것', '같아요', '전생', 

In [20]:
### 중복을 제거 --> set(list) 자료형 생성 --> list(set) 자료형 생성
print(set(words))
print('-'*80)
words = list(set(words))

# 결과 확인하기
print(words)

{'켜놓고', '있어도', '나왔다', '설득', '로', '걸리겠어', '비', '을', '거짓말', '같아', '가끔', '따라', '새', '빠를수록', '좋다', '어서', '전생', '망함', '운동', '좀', '거', '도', '절약', '훈훈해', '잊고', '오늘', '해봐요', '안', '나오세요', '질린다', '은', '좋아요', '만', '마세요', '때', '너무', '또', '나', '집', '필요한', '같아요', '마음', '에', '보인다', '가난한', '좋을', '즐기세요', '결단', '끄고', '뭐', '믿어줘', '자의', '를', '교회', '에는', '가상', '생각', '나라', '그럴', '갔어', '궁금해', '인데', '바빠', '평소', '돈', '설움', '하세요', '생일', '필요했던', '나갔어', '해보세요', '빨리', '난다', '남자친구', '게', '선물', '해', '따뜻하게', '구', '돌아가서', '달', '쫄딱', '출발', '잘생겼어', '땀', '사람', '데려가고', '더', '집착', '함께', '열', '많이', '혼자', '들어올', '나온거', '싶어', '적당히', '하는지', '비싼데', '승진', '불', '식혀주세요', '그', '줄까', '하지', '가만', '가스', '것', '감기', '다시', '좋을까', '켜고', '다음', '사세요', '까지', '화폐', '가', '뭘', '예요', '하셨나요', '인게'}
--------------------------------------------------------------------------------
['켜놓고', '있어도', '나왔다', '설득', '로', '걸리겠어', '비', '을', '거짓말', '같아', '가끔', '따라', '새', '빠를수록', '좋다', '어서', '전생', '망함', '운동', '좀', '거', '도', '절약', '훈훈해', '잊고', '오늘', '해봐요', '안', 

In [21]:
### 특수 토큰 정의

# 변수 설정
PAD = '<PAD>'
SOS = '<SOS>'
EOS = '<EOS>'
UNK = '<UNK>'

# 인덱스 설정
PAD_INDEX = 0
SOS_INDEX = 1
EOS_INDEX = 2
UNK_INDEX = 3

In [22]:
### MARKER 설정
MARKER = [PAD, SOS, EOS, UNK]

In [23]:
### MARKER --> words에 추가 --> 리스트 슬라이싱의 특수한 기능 사용
# words.extend(MARKER)
words[:0] = MARKER

# 결과 확인하기
print(words)

['<PAD>', '<SOS>', '<EOS>', '<UNK>', '켜놓고', '있어도', '나왔다', '설득', '로', '걸리겠어', '비', '을', '거짓말', '같아', '가끔', '따라', '새', '빠를수록', '좋다', '어서', '전생', '망함', '운동', '좀', '거', '도', '절약', '훈훈해', '잊고', '오늘', '해봐요', '안', '나오세요', '질린다', '은', '좋아요', '만', '마세요', '때', '너무', '또', '나', '집', '필요한', '같아요', '마음', '에', '보인다', '가난한', '좋을', '즐기세요', '결단', '끄고', '뭐', '믿어줘', '자의', '를', '교회', '에는', '가상', '생각', '나라', '그럴', '갔어', '궁금해', '인데', '바빠', '평소', '돈', '설움', '하세요', '생일', '필요했던', '나갔어', '해보세요', '빨리', '난다', '남자친구', '게', '선물', '해', '따뜻하게', '구', '돌아가서', '달', '쫄딱', '출발', '잘생겼어', '땀', '사람', '데려가고', '더', '집착', '함께', '열', '많이', '혼자', '들어올', '나온거', '싶어', '적당히', '하는지', '비싼데', '승진', '불', '식혀주세요', '그', '줄까', '하지', '가만', '가스', '것', '감기', '다시', '좋을까', '켜고', '다음', '사세요', '까지', '화폐', '가', '뭘', '예요', '하셨나요', '인게']


In [24]:
### 형태소 리스트를 이용, vocabulary file 생성

vocab_path = 'vocabulary.txt'

with open(vocab_path, 'w', encoding='utf-8') as vocabulary_file:
    for word in words:
        vocabulary_file.write(word+'\n')

In [25]:
### 생성된 단어 사전 불러오기
vocab_path = 'vocabulary.txt'
vocabulary_list = []
with open(vocab_path, 'r', encoding='utf-8') as f:
    for line in f:
        vocabulary_list.append(line.strip())

# 결과 확인하기
print(vocabulary_list)

['<PAD>', '<SOS>', '<EOS>', '<UNK>', '켜놓고', '있어도', '나왔다', '설득', '로', '걸리겠어', '비', '을', '거짓말', '같아', '가끔', '따라', '새', '빠를수록', '좋다', '어서', '전생', '망함', '운동', '좀', '거', '도', '절약', '훈훈해', '잊고', '오늘', '해봐요', '안', '나오세요', '질린다', '은', '좋아요', '만', '마세요', '때', '너무', '또', '나', '집', '필요한', '같아요', '마음', '에', '보인다', '가난한', '좋을', '즐기세요', '결단', '끄고', '뭐', '믿어줘', '자의', '를', '교회', '에는', '가상', '생각', '나라', '그럴', '갔어', '궁금해', '인데', '바빠', '평소', '돈', '설움', '하세요', '생일', '필요했던', '나갔어', '해보세요', '빨리', '난다', '남자친구', '게', '선물', '해', '따뜻하게', '구', '돌아가서', '달', '쫄딱', '출발', '잘생겼어', '땀', '사람', '데려가고', '더', '집착', '함께', '열', '많이', '혼자', '들어올', '나온거', '싶어', '적당히', '하는지', '비싼데', '승진', '불', '식혀주세요', '그', '줄까', '하지', '가만', '가스', '것', '감기', '다시', '좋을까', '켜고', '다음', '사세요', '까지', '화폐', '가', '뭘', '예요', '하셨나요', '인게']


In [26]:
### 단어 사전의 크기 추출
vocab_size = len(vocabulary_list)
print(vocab_size)

125


In [27]:
### 단어 사전 생성 함수 정의
def make_vocabulary(vocabulary_list):
    # 리스트를 이용, 키가 단어이고, 값이 인덱스인 딕셔너리 생성
    word2idx = {word:idx for idx, word in enumerate(vocabulary_list)}
    # 리스트를 이용, 키가 인덱스이고, 값이 단어인 딕셔너리 생성
    idx2word = {idx:word for idx, word in enumerate(vocabulary_list)}
    return word2idx, idx2word

In [28]:
### 단어 사전 생성
word2idx, idx2word = make_vocabulary(vocabulary_list)

# 결과 확인하기
print(f'word2idx : \n{word2idx}')
print('-'*80)
print(f'idx2word : \n{idx2word}')

word2idx : 
{'<PAD>': 0, '<SOS>': 1, '<EOS>': 2, '<UNK>': 3, '켜놓고': 4, '있어도': 5, '나왔다': 6, '설득': 7, '로': 8, '걸리겠어': 9, '비': 10, '을': 11, '거짓말': 12, '같아': 13, '가끔': 14, '따라': 15, '새': 16, '빠를수록': 17, '좋다': 18, '어서': 19, '전생': 20, '망함': 21, '운동': 22, '좀': 23, '거': 24, '도': 25, '절약': 26, '훈훈해': 27, '잊고': 28, '오늘': 29, '해봐요': 30, '안': 31, '나오세요': 32, '질린다': 33, '은': 34, '좋아요': 35, '만': 36, '마세요': 37, '때': 38, '너무': 39, '또': 40, '나': 41, '집': 42, '필요한': 43, '같아요': 44, '마음': 45, '에': 46, '보인다': 47, '가난한': 48, '좋을': 49, '즐기세요': 50, '결단': 51, '끄고': 52, '뭐': 53, '믿어줘': 54, '자의': 55, '를': 56, '교회': 57, '에는': 58, '가상': 59, '생각': 60, '나라': 61, '그럴': 62, '갔어': 63, '궁금해': 64, '인데': 65, '바빠': 66, '평소': 67, '돈': 68, '설움': 69, '하세요': 70, '생일': 71, '필요했던': 72, '나갔어': 73, '해보세요': 74, '빨리': 75, '난다': 76, '남자친구': 77, '게': 78, '선물': 79, '해': 80, '따뜻하게': 81, '구': 82, '돌아가서': 83, '달': 84, '쫄딱': 85, '출발': 86, '잘생겼어': 87, '땀': 88, '사람': 89, '데려가고': 90, '더': 91, '집착': 92, '함께': 93, '열': 94, '많이': 95, '혼자': 96, '

In [29]:
### load_vocabulary() 함수 실행
word2idx, idx2word, vocab_size = load_vocabulary(path=file_path, vocab_path=vocab_path, tokenize_as_morph=True)

In [30]:
### load_vocabulary() 함수 실행의 결과 확인
print(word2idx)

{'<PAD>': 0, '<SOS>': 1, '<EOS>': 2, '<UNK>': 3, '켜놓고': 4, '있어도': 5, '나왔다': 6, '설득': 7, '로': 8, '걸리겠어': 9, '비': 10, '을': 11, '거짓말': 12, '같아': 13, '가끔': 14, '따라': 15, '새': 16, '빠를수록': 17, '좋다': 18, '어서': 19, '전생': 20, '망함': 21, '운동': 22, '좀': 23, '거': 24, '도': 25, '절약': 26, '훈훈해': 27, '잊고': 28, '오늘': 29, '해봐요': 30, '안': 31, '나오세요': 32, '질린다': 33, '은': 34, '좋아요': 35, '만': 36, '마세요': 37, '때': 38, '너무': 39, '또': 40, '나': 41, '집': 42, '필요한': 43, '같아요': 44, '마음': 45, '에': 46, '보인다': 47, '가난한': 48, '좋을': 49, '즐기세요': 50, '결단': 51, '끄고': 52, '뭐': 53, '믿어줘': 54, '자의': 55, '를': 56, '교회': 57, '에는': 58, '가상': 59, '생각': 60, '나라': 61, '그럴': 62, '갔어': 63, '궁금해': 64, '인데': 65, '바빠': 66, '평소': 67, '돈': 68, '설움': 69, '하세요': 70, '생일': 71, '필요했던': 72, '나갔어': 73, '해보세요': 74, '빨리': 75, '난다': 76, '남자친구': 77, '게': 78, '선물': 79, '해': 80, '따뜻하게': 81, '구': 82, '돌아가서': 83, '달': 84, '쫄딱': 85, '출발': 86, '잘생겼어': 87, '땀': 88, '사람': 89, '데려가고': 90, '더': 91, '집착': 92, '함께': 93, '열': 94, '많이': 95, '혼자': 96, '들어올': 97, '나

In [31]:
##### encoder 입력 데이터 생성 함수

MAX_SEQUENCE = 25

def enc_processing(value, dictionary, tokenize_as_morph=False):
    # encoder 입력 데이터 : 단어 사전의 인덱스 값을 가지고 있는 리스트 생성
    encoder_input_index = [] 
    # '질문' --> 한 문장의 길이를 저장할 리스트 생성
    sequences_length = []
    # 형태소 분석기 --> 형태소 분석 실행의 유무 설정
    if tokenize_as_morph:
        value = sentences_tokenized_morphs(value)

    # 한 문장씩 처리
    for sequence in value:
        # 정규 표현식 패턴 생성
        pattern = "[~.,?!\"':;)()]"
        # 정규 표현식을 이용해서 해당 문장 부호를 모두 제거("")
        sequence = re.sub(pattern, "", sequence)
        # 하나의 문장을 정수 인코딩 실행 --> 결과를 저장하기 위한 리스트 생성
        sequence_index = []
        # 문장 --> 공백 단위로 분할 --> 형태소 --> 정수 인코딩 실행
        for word in sequence.split():
            # 형태소(단어) --> dictionary --> index 검색 --> sequence_index에 추가
            if dictionary.get(word) is not None:
                sequence_index.extend([dictionary[word]])
            # 단어가 단어 사전에 존재 하지 않은 경우, UNK 토큰의 인덱스를 넣어 준다 
            else:
                sequence_index.extend([dictionary[UNK]])
        # 문장의 길이를 측정해서 그 값을 sequences_length에 추가
        sequences_length.append(len(sequence_index))
        # 문장의 길이 --> 최대 길이가 길어질 경우 뒤를 자른다
        if len(sequence_index) > MAX_SEQUENCE:
            sequence_index = sequence_index[:MAX_SEQUENCE] 
        # 문장의 길이 --> 최대 길이보다 짧은 경우 PAD 토큰 추가
        sequence_index += (MAX_SEQUENCE - len(sequence_index)) * [dictionary[PAD]]      
        # 한 문장의 처리 결과 --> encoder_input_index 첨가
        encoder_input_index.append(sequence_index)  
    # encoder_input_index를 넘파이 배열로 변환하여 결과값 전달
    return np.array(encoder_input_index), sequences_length

In [32]:
### python dict --> value 추출

'''
1. 변수[key] : 존재하지 않는 key 입력 --> error 발생 
2. 변수.get(key) : 존재하지 않는 key 입력 --> error 발생(X) 
'''

# 변수[key] 
# print(word2idx['이효리'])

# 변수.get(key)
print(word2idx.get('이효리'))

print('-'*80)

# UNK 토큰의 인덱스 확인
print(word2idx['<PAD>'])

None
--------------------------------------------------------------------------------
0


In [33]:
### zero padding 예시
MAX_SEQUENCE = 25
sequence_index = [7, 11, 35, 78, 100]
sequence_index += (MAX_SEQUENCE - len(sequence_index)) * [word2idx['<PAD>']] 

print(sequence_index)

[7, 11, 35, 78, 100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [34]:
### enc_processing() 함수 실행 --> 인코더 입력 데이터 생성
enc_inputs, enc_seq_len = enc_processing(question, word2idx, tokenize_as_morph=True) 

100%|██████████| 20/20 [00:00<00:00, 103.70it/s]


In [35]:
### 인코더 입력 데이터 확인

print(f'enc_inputs : \n{enc_inputs}')
print('-'*80)
print(f'enc_seq_len : \n{enc_seq_len}')

enc_inputs : 
[[ 14  64   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [ 14  53 101  64   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [ 14  34  96 124  18   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [ 48  55  69   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [109   5  88  76   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [ 59 119  85  21   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [110 104 115  73   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [110 104   4  98  13   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [110  10  39  95   6   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [110  10 102 112   9   0   0   0   0   0

In [36]:
##### dec_input_processing() 함수 --> decocer 입력 데이터 생성

def dec_input_processing(value, dictionary, tokenize_as_morph=False):
    # decoder 입력 데이터 : 단어(형태소)의 인덱스로 구성된 리스트 생성
    decoder_input_index=[]
    # '답변' --> 한 문장의 길이를 저장할 리스트 생성
    sequences_length = []
    # 형태소 토크나이저 사용 유무
    if tokenize_as_morph:
        value = sentences_tokenized_morphs(value)

    # 한 문장씩 처리
    for sequence in value:
        # 정규 표현식 패턴 생성
        pattern = "[~.,?!\"':;)()]"
        # 정규 표현식을 이용해서 해당 문장 부호를 제거("")
        sequence = re.sub(pattern, "", sequence)
        # # 문장 --> 단어 사전 --> 정수 인코딩 --> 결과를 저장하기 위한 리스트 생성
        # sequence_index = []
        # 문장 --> SOS + 정수 인코딩 + UNK --> list 덧샘 + list comprehension
        sequence_index = [dictionary[SOS]] + [dictionary[word] if word in dictionary else dictionary[UNK] for word in sequence.split()]
        # 문장의 길이를 측정해서 그 값을 sequences_length에 추가
        sequences_length.append(len(sequence_index))
        # 문장의 길이 --> 최대 길이가 길어질 경우 뒤를 자른다
        if len(sequence_index) > MAX_SEQUENCE:
            sequence_index = sequence_index[:MAX_SEQUENCE] 
        # 문장의 길이 --> 최대 길이보다 짧은 경우 PAD 토큰 추가
        sequence_index += (MAX_SEQUENCE - len(sequence_index)) * [dictionary[PAD]]
        # 한 문장의 처리 결과 --> decoder_input_index 첨가
        decoder_input_index.append(sequence_index)  
    # decoder_input_index를 넘파이 배열로 변환하여 결과값 전달
    return np.array(decoder_input_index), sequences_length

In [37]:
### dec_input_processing() 함수 실행 --> decocer 입력 데이터 생성
dec_inputs, dec_seq_len = dec_input_processing(answer, word2idx, tokenize_as_morph=True)

100%|██████████| 20/20 [00:00<00:00, 78.33it/s]


In [38]:
### decoder 입력 데이터 생성 결과 확인
print(f'dec_inputs : \n{dec_inputs}')
print('-'*80)
print(f'dec_seq_len : \n{dec_seq_len}')

dec_inputs : 
[[  1 106  89  25  62  24 122   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  1 106  89  25  62  24 122   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  1  96  56  50   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  1  68  34 113  97  24 122   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  1  88  11 105   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  1  19  28  16  86  70   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  1  75  42  46  83  52  32   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  1  75  42  46  83  52  32   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  1 116  84  58  91  26  30   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  1  81 117   0   0   0   0   0   0   0

In [39]:
##### decoder 정답 데이터 생성 함수

def dec_target_processing(value, dictionary, tokenize_as_morph=False):
    # decoder 정답 데이터 : 단어 사전 인덱스로 인코딩 된 리스트 생성
    decoder_target_index = []
    # 형태소 토크나이저 사용 유무
    if tokenize_as_morph:
        value = sentences_tokenized_morphs(value)
    # 한 문장씩 처리
    for sequence in value:
        # 정규 표현식 패턴 생성
        pattern = "[~.,!?\"':;)()]"
        # 정규 표현식을 이용해서 문장 부호를 모두 제거("")
        sequence = re.sub(pattern, "", sequence)
        # 문장 --> 정수 인코딩 --> [단어사전[단어] + 단어사전[UNK]] + [단어사전[EOS]] --> list 덧샘 + list comprehension
        sequence_index = [dictionary[word] if word in dictionary else dictionary[UNK] for word in sequence.split()] 
        # 문장의 길이가 최대 길이보다 긴 경우 뒤를 자르고 EOS 토큰을 넣어 준다
        if len(sequence_index) >= MAX_SEQUENCE:
            sequence_index = sequence_index[:MAX_SEQUENCE-1] + [dictionary[EOS]]
        else:
            # 문장의 길이가 최대 길이보다 짧은 경우, 끝에 EOS 토큰 첨가
            sequence_index += [dictionary[EOS]] 
        # 끝에 EOS 토큰 첨가한 후 문장의 길이가 최대 길이보다 짧은 경우 PAD 토큰 추가
        sequence_index += (MAX_SEQUENCE - len(sequence_index))*[dictionary[PAD]]
        # 한 문장에 대해서 정수 인코딩한 결과를 decoder_target_index에 첨가
        decoder_target_index.append(sequence_index)  
    return np.array(decoder_target_index)

In [40]:
### dec_target_processing() 함수 실행 --> decoer 정답 데이터 생성
dec_targets = dec_target_processing(answer, word2idx, tokenize_as_morph=True)

100%|██████████| 20/20 [00:00<00:00, 125.24it/s]


In [41]:
### 결과 확인하기
print(f'decoder 정답 : \n{dec_targets}')

decoder 정답 : 
[[106  89  25  62  24 122   2   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [106  89  25  62  24 122   2   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [ 96  56  50   2   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [ 68  34 113  97  24 122   2   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [ 88  11 105   2   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [ 19  28  16  86  70   2   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [ 75  42  46  83  52  32   2   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [ 75  42  46  83  52  32   2   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [116  84  58  91  26  30   2   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [ 81 117   2   0   0   0   0   0   0   0

## 결과 저장하기

In [42]:
### 데이터 정보 저장하기
data_configs = {}
data_configs['word2idx'] = word2idx
data_configs['idx2word'] = idx2word
data_configs['vocab_size'] = vocab_size
data_configs['pad_symbol'] = PAD
data_configs['sos_symbol'] = SOS
data_configs['eos_symbol'] = EOS
data_configs['unk_symbol'] = UNK

# 결과 확인하기
print(data_configs)

{'word2idx': {'<PAD>': 0, '<SOS>': 1, '<EOS>': 2, '<UNK>': 3, '켜놓고': 4, '있어도': 5, '나왔다': 6, '설득': 7, '로': 8, '걸리겠어': 9, '비': 10, '을': 11, '거짓말': 12, '같아': 13, '가끔': 14, '따라': 15, '새': 16, '빠를수록': 17, '좋다': 18, '어서': 19, '전생': 20, '망함': 21, '운동': 22, '좀': 23, '거': 24, '도': 25, '절약': 26, '훈훈해': 27, '잊고': 28, '오늘': 29, '해봐요': 30, '안': 31, '나오세요': 32, '질린다': 33, '은': 34, '좋아요': 35, '만': 36, '마세요': 37, '때': 38, '너무': 39, '또': 40, '나': 41, '집': 42, '필요한': 43, '같아요': 44, '마음': 45, '에': 46, '보인다': 47, '가난한': 48, '좋을': 49, '즐기세요': 50, '결단': 51, '끄고': 52, '뭐': 53, '믿어줘': 54, '자의': 55, '를': 56, '교회': 57, '에는': 58, '가상': 59, '생각': 60, '나라': 61, '그럴': 62, '갔어': 63, '궁금해': 64, '인데': 65, '바빠': 66, '평소': 67, '돈': 68, '설움': 69, '하세요': 70, '생일': 71, '필요했던': 72, '나갔어': 73, '해보세요': 74, '빨리': 75, '난다': 76, '남자친구': 77, '게': 78, '선물': 79, '해': 80, '따뜻하게': 81, '구': 82, '돌아가서': 83, '달': 84, '쫄딱': 85, '출발': 86, '잘생겼어': 87, '땀': 88, '사람': 89, '데려가고': 90, '더': 91, '집착': 92, '함께': 93, '열': 94, '많이': 95, '혼자': 96, 

In [43]:
### 결과 저장하기
np.save('enc_inputs.npy', enc_inputs)
np.save('dec_inputs.npy', dec_inputs)
np.save('dec_targets.npy', dec_targets)
with open('data_configs.json', 'w') as f:
    json.dump(data_configs, f)

In [44]:
### 저장된 결과 불러오기
enc_inputs = np.load('enc_inputs.npy')
dec_inputs = np.load('dec_inputs.npy')
dec_targets = np.load('dec_targets.npy')
with open('data_configs.json', 'r') as f:
    data_confgis = json.load(f)

# 결과 확인하기
print(enc_inputs[0])
print('-'*80)
print(dec_inputs[0])
print('-'*80)
print(dec_targets[0])
print('-'*80)
print(data_configs) 

[14 64  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0]
--------------------------------------------------------------------------------
[  1 106  89  25  62  24 122   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0]
--------------------------------------------------------------------------------
[106  89  25  62  24 122   2   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0]
--------------------------------------------------------------------------------
{'word2idx': {'<PAD>': 0, '<SOS>': 1, '<EOS>': 2, '<UNK>': 3, '켜놓고': 4, '있어도': 5, '나왔다': 6, '설득': 7, '로': 8, '걸리겠어': 9, '비': 10, '을': 11, '거짓말': 12, '같아': 13, '가끔': 14, '따라': 15, '새': 16, '빠를수록': 17, '좋다': 18, '어서': 19, '전생': 20, '망함': 21, '운동': 22, '좀': 23, '거': 24, '도': 25, '절약': 26, '훈훈해': 27, '잊고': 28, '오늘': 29, '해봐요': 30, '안': 31, '나오세요': 32, '질린다': 33, '은': 34, '좋아요': 35, '만': 36, '마세요': 37, '때': 38, '너무': 39, '또': 40, '나': 41, '집': 42, '필요한': 43, '같아요': 44, '마음'

## 전처리 모듈 확인

In [ ]:
from preprocess import *

In [46]:
### 데이터 불러오기
question, answer = load_data(file_path)

# 결과 불러오기
print(f'질문 컬럼 : \n{question}')
print('-'*80)
print(f'답변 컬럼 : \n{answer}')

질문 컬럼 : 
['가끔 궁금해', '가끔 뭐하는지 궁금해', '가끔은 혼자인게 좋다', '가난한 자의 설움', '가만 있어도 땀난다', '가상화폐 쫄딱 망함', '가스불 켜고 나갔어', '가스불 켜놓고 나온거 같아', '가스비 너무 많이 나왔다.', '가스비 비싼데 감기 걸리겠어', '남자친구 교회 데려가고 싶어', '남자친구 또 운동 갔어', '남자친구 생일인데 뭘 줄까', '남자친구 승진 선물로 뭐가 좋을까?', '남자친구 오늘 따라 훈훈해 보인다', '남자친구 오늘 좀 질린다.', '남자친구가 나 안 믿어줘', '남자친구가 너무 바빠', '남자친구가 너무 운동만 해', '남자친구가 너무 잘생겼어']
--------------------------------------------------------------------------------
답변 컬럼 : 
['그 사람도 그럴 거예요.', '그 사람도 그럴 거예요.', '혼자를 즐기세요.', '돈은 다시 들어올 거예요.', '땀을 식혀주세요.', '어서 잊고 새출발 하세요.', '빨리 집에 돌아가서 끄고 나오세요.', '빨리 집에 돌아가서 끄고 나오세요.', '다음 달에는 더 절약해봐요.', '따뜻하게 사세요!', '마음을 열 때까지 설득해보세요.', '운동을 함께 해보세요.', '평소에 필요한 것 생각해보세요.', '평소에 필요했던 게 좋을 것 같아요.', '전생에 나라를 구하셨나요.', '결단은 빠를수록 좋아요.', '거짓말 적당히 하세요.', '너무 집착하지 마세요.', '운동을 함께 해보세요.', '전생에 나라를 구하셨나요.']


In [47]:
### 단어 사전 생성하기
file_path = 'ChatBotData.csv_short'
vocab_path = 'vocabulary.txt'

word2idx, idx2word, vocab_size = load_vocabulary(path=file_path, vocab_path=vocab_path, tokenize_as_morph=True)

In [48]:
### 단어 사전 생성 결과 확인하기
print(word2idx)
print('-'*80)
print(idx2word)
print('-'*80)
print(vocab_size)

{'<PAD>': 0, '<SOS>': 1, '<EOS>': 2, '<UNK>': 3, '켜놓고': 4, '있어도': 5, '나왔다': 6, '설득': 7, '로': 8, '걸리겠어': 9, '비': 10, '을': 11, '거짓말': 12, '같아': 13, '가끔': 14, '따라': 15, '새': 16, '빠를수록': 17, '좋다': 18, '어서': 19, '전생': 20, '망함': 21, '운동': 22, '좀': 23, '거': 24, '도': 25, '절약': 26, '훈훈해': 27, '잊고': 28, '오늘': 29, '해봐요': 30, '안': 31, '나오세요': 32, '질린다': 33, '은': 34, '좋아요': 35, '만': 36, '마세요': 37, '때': 38, '너무': 39, '또': 40, '나': 41, '집': 42, '필요한': 43, '같아요': 44, '마음': 45, '에': 46, '보인다': 47, '가난한': 48, '좋을': 49, '즐기세요': 50, '결단': 51, '끄고': 52, '뭐': 53, '믿어줘': 54, '자의': 55, '를': 56, '교회': 57, '에는': 58, '가상': 59, '생각': 60, '나라': 61, '그럴': 62, '갔어': 63, '궁금해': 64, '인데': 65, '바빠': 66, '평소': 67, '돈': 68, '설움': 69, '하세요': 70, '생일': 71, '필요했던': 72, '나갔어': 73, '해보세요': 74, '빨리': 75, '난다': 76, '남자친구': 77, '게': 78, '선물': 79, '해': 80, '따뜻하게': 81, '구': 82, '돌아가서': 83, '달': 84, '쫄딱': 85, '출발': 86, '잘생겼어': 87, '땀': 88, '사람': 89, '데려가고': 90, '더': 91, '집착': 92, '함께': 93, '열': 94, '많이': 95, '혼자': 96, '들어올': 97, '나

In [49]:
### encoder 입력 데이터 생성
enc_inputs, enc_seq_len = enc_processing(question, word2idx, tokenize_as_morph=True)

100%|██████████| 20/20 [00:00<00:00, 407.88it/s]


In [50]:
### 생성된 encoder 입력 데이터 확인
print(enc_inputs)
print('-'*80)
print(enc_seq_len)

[[ 14  64   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [ 14  53 101  64   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [ 14  34  96 124  18   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [ 48  55  69   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [109   5  88  76   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [ 59 119  85  21   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [110 104 115  73   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [110 104   4  98  13   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [110  10  39  95   6   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [110  10 102 112   9   0   0   0   0   0   0   0   0  

In [51]:
### decoder 입력 데이터 생성
dec_inputs, dec_seq_len = dec_input_processing(answer, word2idx, tokenize_as_morph=True)

100%|██████████| 20/20 [00:00<00:00, 393.22it/s]


In [52]:
### 생성된 decoder 입력 데이터 확인
print(dec_inputs)
print('-'*80)
print(dec_seq_len)

[[  1 106  89  25  62  24 122   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  1 106  89  25  62  24 122   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  1  96  56  50   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  1  68  34 113  97  24 122   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  1  88  11 105   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  1  19  28  16  86  70   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  1  75  42  46  83  52  32   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  1  75  42  46  83  52  32   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  1 116  84  58  91  26  30   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  1  81 117   0   0   0   0   0   0   0   0   0   0  

In [53]:
### decoder 정답 데이터 생성
enc_targets = dec_target_processing(answer, word2idx, tokenize_as_morph=True)

# 결과 확인하기
print(enc_targets)

100%|██████████| 20/20 [00:00<00:00, 449.19it/s]

[[106  89  25  62  24 122   2   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [106  89  25  62  24 122   2   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [ 96  56  50   2   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [ 68  34 113  97  24 122   2   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [ 88  11 105   2   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [ 19  28  16  86  70   2   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [ 75  42  46  83  52  32   2   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [ 75  42  46  83  52  32   2   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [116  84  58  91  26  30   2   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [ 81 117   2   0   0   0   0   0   0   0   0   0   0  